# Blog Components

> Reusable components for the technical blog

In [ ]:
#| default_exp blog_components

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
from fasthtml.common import *
from monsterui.all import *
from typing import List, Dict, Any

__all__ = ['create_nav', 'topic_card', 'math_block', 'code_block']

## Navigation Components

In [ ]:
#| export
def create_nav(topic: str = None) -> NavBar:
    "Create main navigation with MonsterUI `NavBar` for current `topic`"
    nav_items = [
        ('Home', '/'),
        ('RBE Series', '/rbe/'),
        ('Future Topics', '/topics/'),
        ('About', '/about/')
    ]
    
    # Create navigation links
    nav_links = []
    for title, url in nav_items:
        # Check if this item should be active
        is_active = False
        if topic:
            if title.lower().startswith(topic.lower()) or \
               (topic.lower() == "home" and title == "Home") or \
               (topic.lower() == "rbe" and title == "RBE Series"):
                is_active = True
        
        # Add styling for active state
        link_cls = "nav-link font-medium px-3 py-2 rounded-md transition-colors"
        if is_active:
            link_cls += " bg-primary text-primary-foreground"
        else:
            link_cls += " text-muted-foreground hover:text-foreground hover:bg-muted"
            
        nav_links.append(A(title, href=url, cls=link_cls))
    
    # Add MonsterUI ThemePicker (mode only for dark/light toggle)
    theme_toggle = ThemePicker(
        color=False,
        radii=False, 
        shadows=False,
        font=False,
        mode=True,
        cls="p-2"
    )
    
    # Create brand/title
    brand = Div(
        H3("Matthew Redrup's Blog", cls="text-lg font-bold text-foreground"),
        P("Ramblings on AI & Cybersecurity", cls="text-sm text-muted-foreground hidden sm:block"),
        cls="flex flex-col"
    )
    
    return NavBar(
        *nav_links,
        theme_toggle,
        brand=brand,
        cls="border-b"
    )

In [ ]:
# Test create_nav - simplified for debugging
nav = create_nav("home")
print(f"NavBar type: {type(nav)}")
print(f"NavBar children count: {len(nav.children)}")
for i, child in enumerate(nav.children):
    print(f"Child {i}: {type(child).__name__} - {getattr(child, 'cls', 'no cls')}")

# Basic test that it's a NavBar
assert hasattr(nav, 'brand'), "Should have brand attribute"
assert hasattr(nav, 'children'), "Should have children attribute"

In [ ]:
#| export
def topic_card(title: str, desc: str, url: str, status: str = "available") -> Div:
    "Create a card for blog `title` with `desc`ription and `url`"
    status_cls = f"topic-{status}"
    
    return Div(
        H3(title),
        P(desc),
        A("Read More →", href=url, cls="topic-link") if status == "available" else Span("Coming Soon", cls="coming-soon"),
        cls=f"topic-card {status_cls}"
    )

In [ ]:
# Test topic_card
card = topic_card("Test Topic", "A test description", "/test/", "available")
test_eq(card.children[0].children[0], "Test Topic")
test_eq(card.children[1].children[0], "A test description")
test_eq(card.children[2].href, "/test/")

# Test coming soon status
card_soon = topic_card("Future", "Coming later", "/future/", "coming soon")
test_eq(card_soon.children[2].children[0], "Coming Soon")

In [ ]:
#| export
def math_block(tex: str, block: bool = True) -> Div:
    "Render LaTeX `tex` using KatexMarkdownJS"
    delim = "$$" if block else "$"
    content = f"{delim}{tex}{delim}"
    
    return Div(
        content,
        cls="marked math-content" if block else "marked math-inline"
    )

In [ ]:
# Test math_block
math = math_block("x^2 + y^2 = z^2")
test_eq(math.children[0], "$$x^2 + y^2 = z^2$$")
test_eq(math.cls, "marked math-content")

# Test inline math
inline = math_block("E = mc^2", block=False)
test_eq(inline.children[0], "$E = mc^2$")
test_eq(inline.cls, "marked math-inline")

In [ ]:
# Test code_block
cb = code_block("print('hello')", "python")
test_eq(cb.children[0].children[0].children[0], "print('hello')")
test_eq(cb.children[0].children[0].cls, "language-python")

# Test with title
cb_titled = code_block("const x = 42;", "javascript", "Example Code")
test_eq(len(cb_titled.children), 2)
test_eq(cb_titled.children[0].children[0], "Example Code")

## Code Display Components

In [ ]:
#| export
def code_block(code: str, language: str = "python", title: str = None) -> Div:
    """Create a syntax-highlighted code block using FastHTML's HighlightJS"""
    code_element = Pre(
        Code(code, cls=f"language-{language}")
    )
    
    if title:
        return Div(
            Div(title, cls="code-title"),
            code_element,
            cls="code-block-container"
        )
    else:
        return Div(code_element, cls="code-block-container")